#### Calculates mean, median, and std dev of coherence of subsequent pairs

In [1]:
# all the good stuff
import os,sys
import glob
import pandas as pd
import math
from osgeo import gdal
import numpy as np
from array import array
from scipy  import signal
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from scipy.stats import gaussian_kde
#matplotlib  widget

#### <span style="color:red"> Edit this cell for path, point of interest, full res, and downlook dims</span>

In [ ]:
workdir = '/data/krd86/cropped'
alooks = 4 # azimuth looks
rlooks = 20 # range looks
rx = rlooks # uses these for the window 
ry = alooks

# size of full res SLCs
dx=30000
dy=3500

# downlooked size, uses this to save files
dldx = dx/rlooks
dldy = dy/alooks

#### Load in dates

In [2]:
# load in dates
dates = [os.path.basename(x) for x in glob.glob(workdir+"SLC_vv/2*")]
dates = sorted(dates)
nd = len(dates)

#### Define window for spatial averaging through convolution

In [4]:
# load in 4 rows at a time (range), convolve over all columns (azimuth), average over columns, take downlooked coords

# defines window of 20rx4a to convolve with full res SLC, zero padded
windx = np.zeros((1,rx*2+1))
windx[0,rx-math.floor(rx/2) + np.arange(1,rx)]=1
windy = np.zeros((1,ry*2+1))
windy[0,ry-math.floor(ry/2) + np.arange(1,ry)]=1
windx = windx/np.sum(windx)
windy = windy/np.sum(windy)
wind2d = windy.T*windx

#### Convolve window with subsquent pairs of VV SLCs strip by strip

In [35]:
slc1 = np.ndarray([alooks,dx],'complex')
slc2 = np.ndarray([alooks,dx],'complex')
cors = np.ndarray([int(dy/alooks),int(dx/rlooks),nd-1],'float')

for i in np.arange(nd-1): #loop through subsequent SLC pairs
    ds1 = gdal.Open(workdir+"SLC_vv/"+dates[i]+"/"+dates[i]+".slc.full", gdal.GA_ReadOnly)
    ds2 = gdal.Open(workdir+"SLC_vv/"+dates[i+1]+"/"+dates[i+1]+".slc.full", gdal.GA_ReadOnly)
    for j in range(0,dy-1,alooks+1): #loop over 4x30000 strips
        slc1[:,:] = ds1.GetRasterBand(1).ReadAsArray(0,j,dx,alooks)
        slc2[:,:] = ds2.GetRasterBand(1).ReadAsArray(0,j,dx,alooks)
        
        a   = slc1*np.conj(slc1)
        b   = slc2*np.conj(slc2)
        c   = slc1*np.conj(slc2)
        ca  = abs(c) 
            
        asum = signal.convolve2d(a, wind2d, mode='same') 
        bsum = signal.convolve2d(b, wind2d, mode='same')
        csum = signal.convolve2d(c, wind2d, mode='same')
        cmag = signal.convolve2d(ca, wind2d, mode='same')
        
        #cpx3 = csum/np.sqrt(asum*bsum) #alternative def of coherence
        cpx3 = csum/cmag
        cpx3 = np.mean(cpx3,axis=0)
        sm   = abs(cpx3)
        sm[np.isnan(sm)] = 0;
        
        if j==5:
            cors[math.floor(j/alooks),:,i] = sm[10::20]
        else:    
            cors[math.ceil(j/alooks),:,i] = sm[10::20]
    
    good        = cors>0
    cors[~good] = np.nan
    print('finished '+str(i+1)+'th igram')

finished 1th igram
finished 2th igram
finished 3th igram
finished 4th igram
finished 5th igram
finished 6th igram
finished 7th igram
finished 8th igram
finished 9th igram
finished 10th igram
finished 11th igram
finished 12th igram
finished 13th igram
finished 14th igram
finished 15th igram
finished 16th igram
finished 17th igram
finished 18th igram
finished 19th igram
finished 20th igram
finished 21th igram
finished 22th igram
finished 23th igram
finished 24th igram
finished 25th igram
finished 26th igram
finished 27th igram
finished 28th igram
finished 29th igram
finished 30th igram
finished 31th igram
finished 32th igram
finished 33th igram
finished 34th igram
finished 35th igram
finished 36th igram
finished 37th igram
finished 38th igram
finished 39th igram
finished 40th igram
finished 41th igram
finished 42th igram
finished 43th igram
finished 44th igram
finished 45th igram
finished 46th igram
finished 47th igram
finished 48th igram
finished 49th igram
finished 50th igram
finished 

#### Calculate mean, median, and std dev

In [ ]:
#compute mean, median, and std dev
cor_avg = np.mean(cors,axis=2)
cor_med = np.median(cors,axis=2)
cor_std = np.std(cors,axis=2)

#### Save stats as binary files

In [ ]:
driver=gdal.GetDriverByName('ISCE')
colds = driver.Create(workdir+'cor_avg.r4',1500,875,1,gdal.GDT_Float32)
colds.GetRasterBand(1).WriteArray(cor_avg)
colds=None

colds = driver.Create(workdir+'cor_med.r4',1500,875,1,gdal.GDT_Float32)
colds.GetRasterBand(1).WriteArray(cor_med)
colds=None

colds = driver.Create(workdir+'cor_std.r4',1500,875,1,gdal.GDT_Float32)
colds.GetRasterBand(1).WriteArray(cor_std)
colds=None